# Inference in a Propositional Knowledge Base

This Jupyter notebook demonstrates how to create a knowledge base for propositional logic and how to apply different inference algorithms to ask questions to the knowledge base.

Import modules:

In [1]:
from utils import *
from logic import *
from notebook import psource

## Wumpus World Knowledge Base

We first construct the knowledge base for the Wumpus World example from the lecture. The proposition symbols are:

<br/>
$P_{y, x}$ is true if there is a pit at position [y,x].<br/>
$B_{y, x}$ is true if the agent senses breeze at position [y,x].<br/>

We create all required proposition symbols with the `symbols` function:

In [2]:
(P11,P12,P13,P21,P22,P31,B11,B12,B21) = symbols('P11, P12, P13, P21, P22, P31, B11, B12, B21')

Next, we create an empty propositional knowledge base by calling the constructor of the class `PropKB`:

In [3]:
wumpus_kb = PropKB()

Now we can add all the knowledge we have about the Wumpus World environment to our knowledge base:

There is no pit in `[1,1]`.

In [4]:
wumpus_kb.tell(~P11)

A square is breezy if and only if there is a pit in a neighboring square. This has to be stated for each square but for now, we include just the relevant squares.

In [5]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

Finally, we include the breeze percepts for the first two squares:

In [6]:
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

We can check the clauses stored in a knowledge base by accessing its `clauses` property:

In [7]:
wumpus_kb.clauses

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

We see that some equivalences automatically got converted into two implications which afterwards got converted to Conjunctive Normal Form (CNF).

### Inference by Enumeration
Given our knowledge base descibing the Wumpus World environment we now want to infere if the adjacent squares are safe. To solve this problem, we can use logical inference. The inference algorithm that is considered first is *Truth Table Enumeration*. To deside if a sentence $\alpha$ is entailed by the knowledge base, *Truth Table Enumeration* enumerates all models and checks if $\alpha$ is true for all models where the knowledge base is true. The algorithm is implemented in the function `tt_entails`:

In [8]:
psource(tt_entails)

Internally, the function `tt_entails` calls the function `tt_check_all` to check if the sentence $\alpha$ and the knowledge base are true for a certain model:

In [9]:
psource(tt_check_all)

Before we apply *Truth Table Enumeration* to our Wumpus World knowledge base we demonstrate the usage of the function `tt_entails` on some simple examples. Given two proposition symbols $P$ and $Q$, let us consider the knowledge base $KB = P \wedge Q$ and the sentence $\alpha = Q$. Since $P \wedge Q$ can only become true if $Q$ is true, the knowledge base obviously entails the sentence $\alpha$:

In [10]:
(P,Q) = symbols('P, Q')
tt_entails(P & Q, Q)

True

If we instead consider the knowledge base $KB = P \vee Q$, the sentence $\alpha$ is not entailed by the knowledge base:

In [11]:
tt_entails(P | Q, Q)

False

Now let's come back to our Wumpus World problem. The function `tt_entails()` takes an object of the class `Expr` which is a conjunction of clauses as the input instead of the `KB` itself. Instead of manually converting our knowledge base we can use the `ask_if_true` method of the `PropKB` class which does all the required conversions automatically. Let's check if there is a pit in the square [1,2]:

In [12]:
wumpus_kb.ask_if_true(~P12)

True

### Proof by Resolution


One alternative to the computational expensive *Truth Table Enumeration* is to proof the entailment using resolution. The algorithm that implements proof by resolution requires a knowledge base in Conjunctive Normal Form (CNF). We can apply the function `to_cnf` to convert logical sentences to CNF:

In [13]:
psource(to_cnf)

Here some examples that demsonstrate the automated conversion to CNF:

In [14]:
(A,B,C,D) = symbols('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [15]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

The resolution algorithm, which is implemented by the function `pl_resolution`, internally utilizes the function `to_cnf` to convert the knowledge base to CNF:

In [16]:
psource(pl_resolution)

Let's come back to our Wumpus World example. As done in the lecture slides, we first add some additional facts to our knowledge base:

There is no breeze in square [1,2]:

In [17]:
wumpus_kb.tell(~B12)

A square is breezy if and only if there is a pit in a neighboring square:

In [18]:
wumpus_kb.tell(B12 | '<=>' | ((P11 | P22 | P13)))

Now we apply the resolution algorithm to check if there is a pit in the square [3,1]:

In [19]:
pl_resolution(wumpus_kb, P31)

True